<a href="https://colab.research.google.com/github/azganushpoghosyan/mit_assignments/blob/main/ps3_hand_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random
import string

In [ ]:
VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

In [ ]:
SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

In [ ]:
WORDLIST_FILENAME = "words.txt"

###Helping Functions

In [ ]:
def load_words():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.read()
    # wordlist: list of strings
    wordlist = line.split()
    print("  ", len(wordlist), "words loaded.")
    return wordlist


In [ ]:
wordlist = [x.lower() for x in load_words()]

Loading word list from file...
   83667 words loaded.


In [ ]:
! python ps3.py

Loading word list from file...
   83667 words loaded.
play_game not implemented.


In [ ]:
def get_frequency_dict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq
	

###Problem 1

In [ ]:
def get_word_score(word, n):
    """
    Returns the score for a word. Assumes the word is a
    valid word.

    You may assume that the input word is always either a string of letters, 
    or the empty string "". You may not assume that the string will only contain 
    lowercase letters, so you will have to handle uppercase and mixed case strings 
    appropriately. 

	The score for a word is the product of two components:

	The first component is the sum of the points for letters in the word.
	The second component is the larger of:
            1, or
            7*wordlen - 3*(n-wordlen), where wordlen is the length of the word
            and n is the hand length when the word was played

	Letters are scored as in Scrabble; A is worth 1, B is
	worth 3, C is worth 3, D is worth 2, E is worth 1, and so on.

    word: string
    n: int >= 0
    returns: int >= 0
    """
    word_lower = word.lower()
    first_component = 0
    for i in word_lower:
      if i == '*':
        first_component = first_component
      else:
        first_component = first_component + SCRABBLE_LETTER_VALUES[i]
    if (7 * len(word_lower)) - (3 * (n - len(word_lower))) > 1:
      second_component = 7 * len(word_lower) - 3 * (n - len(word_lower))
    else:
      second_component = 1
    score = first_component * second_component
    return score

###Test Problem 1

In [ ]:
def test_get_word_score():
    """
    Unit test for get_word_score
    """
    failure=False
    # dictionary of words and scores
    words = {("", 7):0, ("it", 7):2, ("was", 7):54, ("weed", 6):176,
             ("scored", 7):351, ("WaYbILl", 7):735, ("Outgnaw", 7):539,
             ("fork", 7):209, ("FORK", 4):308}
    for (word, n) in words.keys():
        score = get_word_score(word, n)
        if score != words[(word, n)]:
            print("FAILURE: test_get_word_score()")
            print("\tExpected", words[(word, n)], "points but got '" + \
                  str(score) + "' for word '" + word + "', n=" + str(n))
            failure=True
    if not failure:
        print("SUCCESS: test_get_word_score()")

In [ ]:
test_get_word_score()

SUCCESS: test_get_word_score()


###Problem 2

In [ ]:
def update_hand(hand, word):
    """
    Does NOT assume that hand contains every letter in word at least as
    many times as the letter appears in word. Letters in word that don't
    appear in hand should be ignored. Letters that appear in word more times
    than in hand should never result in a negative count; instead, set the
    count in the returned hand to 0 (or remove the letter from the
    dictionary, depending on how your code is structured). 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """

    new_hand = hand.copy()
    word_lower = word.lower()
    for i in word_lower:
      if i in new_hand.keys():
        new_hand[i] = new_hand.get(i, 0) - 1
      
    return new_hand

###Test Problem 2

In [ ]:
def test_update_hand():
    """
    Unit test for update_hand
    """
    # test 1
    handOrig = {'a':1, 'q':1, 'l':2, 'm':1, 'u':1, 'i':1}
    handCopy = handOrig.copy()
    word = "quail"

    hand2 = update_hand(handCopy, word)
    expected_hand1 = {'l':1, 'm':1}
    expected_hand2 = {'a':0, 'q':0, 'l':1, 'm':1, 'u':0, 'i':0}
    if hand2 != expected_hand1 and hand2 != expected_hand2:
        print("FAILURE: test_update_hand('"+ word +"', " + str(handOrig) + ")")
        print("\tReturned: ", hand2, "\n\t-- but expected:", expected_hand1, "or", expected_hand2)

        return # exit function
    if handCopy != handOrig:
        print("FAILURE: test_update_hand('"+ word +"', " + str(handOrig) + ")")
        print("\tOriginal hand was", handOrig)
        print("\tbut implementation of update_hand mutated the original hand!")
        print("\tNow the hand looks like this:", handCopy)
        
        return # exit function
        
    # test 2
    handOrig = {'e':1, 'v':2, 'n':1, 'i':1, 'l':2}
    handCopy = handOrig.copy()
    word = "Evil"

    hand2 = update_hand(handCopy, word)
    expected_hand1 = {'v':1, 'n':1, 'l':1}
    expected_hand2 = {'e':0, 'v':1, 'n':1, 'i':0, 'l':1}
    if hand2 != expected_hand1 and hand2 != expected_hand2:
        print("FAILURE: test_update_hand('"+ word +"', " + str(handOrig) + ")")        
        print("\tReturned: ", hand2, "\n\t-- but expected:", expected_hand1, "or", expected_hand2)

        return # exit function

    if handCopy != handOrig:
        print("FAILURE: test_update_hand('"+ word +"', " + str(handOrig) + ")")
        print("\tOriginal hand was", handOrig)
        print("\tbut implementation of update_hand mutated the original hand!")
        print("\tNow the hand looks like this:", handCopy)
        
        return # exit function

    # test 3
    handOrig = {'h': 1, 'e': 1, 'l': 2, 'o': 1}
    handCopy = handOrig.copy()
    word = "HELLO"

    hand2 = update_hand(handCopy, word)
    expected_hand1 = {}
    expected_hand2 = {'h': 0, 'e': 0, 'l': 0, 'o': 0}
    if hand2 != expected_hand1 and hand2 != expected_hand2:
        print("FAILURE: test_update_hand('"+ word +"', " + str(handOrig) + ")")                
        print("\tReturned: ", hand2, "\n\t-- but expected:", expected_hand1, "or", expected_hand2)
        
        return # exit function

    if handCopy != handOrig:
        print("FAILURE: test_update_hand('"+ word +"', " + str(handOrig) + ")")
        print("\tOriginal hand was", handOrig)
        print("\tbut implementation of update_hand mutated the original hand!")
        print("\tNow the hand looks like this:", handCopy)
        
        return # exit function

    print("SUCCESS: test_update_hand()")

In [ ]:
test_update_hand()

SUCCESS: test_update_hand()


###Problem 3

In [ ]:
def is_valid_word(word, hand, wordlist):
    """
    Returns True if word is in the wordlist and is entirely
    composed of letters in the hand. Otherwise, returns False.
    Does not mutate hand or wordlist.
   
    word: string
    hand: dictionary (string -> int)
    wordlist: list of lowercase strings
    returns: boolean
    """
    new_hand = hand.copy()
    word_lower = word.lower()
    checklist = []

    if word_lower not in wordlist:
      return False
    else:
      for i in word_lower:
        if i in new_hand.keys() and new_hand[i] > 0:
          new_hand = update_hand(new_hand, i)
          checklist.append('true')
        else:
          checklist.append('false')
      if len(set(checklist)) == 1 and checklist[0] == 'true':
        return True
      else: 
        return False 
      
   

###Test Problem 3

In [ ]:
def test_is_valid_word(word_list):
    """
    Unit test for is_valid_word
    """
    failure=False
    # test 1
    word = "hello"
    handOrig = get_frequency_dict(word)
    handCopy = handOrig.copy()

    if not is_valid_word(word, handCopy, word_list):
        print("FAILURE: test_is_valid_word()")
        print("\tExpected True, but got False for word: '" + word + "' and hand:", handOrig)

        failure = True

    # Test a second time to see if word_list or hand has been modified
    if not is_valid_word(word, handCopy, word_list):
        print("FAILURE: test_is_valid_word()")

        if handCopy != handOrig:
            print("\tTesting word", word, "for a second time - be sure you're not modifying hand.")
            print("\tAt this point, hand ought to be", handOrig, "but it is", handCopy)

        else:
            print("\tTesting word", word, "for a second time - have you modified word_list?")
            wordInWL = word in word_list
            print("The word", word, "should be in word_list - is it?", wordInWL)

        print("\tExpected True, but got False for word: '" + word + "' and hand:", handCopy)

        failure = True


    # test 2
    hand = {'r': 1, 'a': 3, 'p': 2, 'e': 1, 't': 1, 'u':1}
    word = "Rapture"

    if  is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word()")
        print("\tExpected False, but got True for word: '" + word + "' and hand:", hand)

        failure = True        

    # test 3
    hand = {'n': 1, 'h': 1, 'o': 1, 'y': 1, 'd':1, 'w':1, 'e': 2}
    word = "honey"

    if  not is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word()")
        print("\tExpected True, but got False for word: '"+ word +"' and hand:", hand)

        failure = True                        

    # test 4
    hand = {'r': 1, 'a': 3, 'p': 2, 't': 1, 'u':2}
    word = "honey"

    if  is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word()")
        print("\tExpected False, but got True for word: '" + word + "' and hand:", hand)
        
        failure = True

    # test 5
    hand = {'e':1, 'v':2, 'n':1, 'i':1, 'l':2}
    word = "EVIL"
    
    if  not is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word()")
        print("\tExpected True, but got False for word: '" + word + "' and hand:", hand)
        
        failure = True
        
    # test 6
    word = "Even"

    if  is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word()")
        print("\tExpected False, but got True for word: '" + word + "' and hand:", hand)
        print("\t(If this is the only failure, make sure is_valid_word() isn't mutating its inputs)")        
        
        failure = True        

    if not failure:
        print("SUCCESS: test_is_valid_word()")

In [ ]:
test_is_valid_word(wordlist)

SUCCESS: test_is_valid_word()


###Problem 4

In [ ]:
def deal_hand(n):
    """
    Returns a random hand containing n lowercase letters.
    ceil(n/3) letters in the hand should be VOWELS (note,
    ceil(n/3) means the smallest integer not less than n/3).

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    
    hand={}
    num_vowels = int(math.ceil(n / 3))

    for i in range(num_vowels - 1):
        x = random.choice(VOWELS)
        hand[x] = hand.get(x, 0) + 1
        hand['*'] = 1
    
    for i in range(num_vowels, n):    
        x = random.choice(CONSONANTS)
        hand[x] = hand.get(x, 0) + 1
    
    return hand


In [ ]:
def is_valid_word(word, hand, wordlist):
    """
    Returns True if word is in the word_list and is entirely
    composed of letters in the hand. Otherwise, returns False.
    Does not mutate hand or word_list.
   
    word: string
    hand: dictionary (string -> int)
    word_list: list of lowercase strings
    returns: boolean
    """
    new_hand = hand.copy()
    checklist = []
    word_lower = word.lower()

    if '*' in word_lower:
      possible_words = []
      for i in VOWELS:
        possible_words.append(word_lower.replace('*', i))
      real_words = []
      for any_word in possible_words:
        if any_word in wordlist:
          real_words.append(any_word)
      if len(real_words) == 0:
        return False 
      else: 
        for i in word_lower:
          if i in new_hand.keys() and new_hand[i] > 0: 
            new_hand = update_hand(new_hand, i)
            checklist.append('true')
          else:
            checklist.append('false')
        if len(set(checklist)) == 1 and checklist[0] == 'true':
          return True
        else: 
          return False 
    else:
      if word_lower not in wordlist:
        return False
      else:
        for i in word_lower:
          if i in new_hand.keys() and new_hand[i] > 0:
            new_hand = update_hand(new_hand, i)
            checklist.append('true')
          else:
            checklist.append('false')
        if len(set(checklist)) == 1 and checklist[0] == 'true':
          return True
        else: 
          return False 


###Test Problem 4

In [ ]:
def test_wildcard(word_list):
    """
    Unit test for is_valid_word
    """
    failure=False

    # test 1
    hand = {'a': 1, 'r': 1, 'e': 1, 'j': 2, 'm': 1, '*': 1}
    word = "e*m"

    if is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word() with wildcards")
        print("\tExpected False, but got True for word: '" + word + "' and hand:", hand)

        failure = True

    # test 2
    hand = {'n': 1, 'h': 1, '*': 1, 'y': 1, 'd':1, 'w':1, 'e': 2}
    word = "honey"

    if is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word() with wildcards")
        print("\tExpected False, but got True for word: '"+ word +"' and hand:", hand)

        failure = True

    # test 3
    hand = {'n': 1, 'h': 1, '*': 1, 'y': 1, 'd':1, 'w':1, 'e': 2}
    word = "h*ney"

    if not is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word() with wildcards")
        print("\tExpected True, but got False for word: '"+ word +"' and hand:", hand)

        failure = True

    # test 4
    hand = {'c': 1, 'o': 1, '*': 1, 'w': 1, 's':1, 'z':1, 'y': 2}
    word = "c*wz"

    if is_valid_word(word, hand, word_list):
        print("FAILURE: test_is_valid_word() with wildcards")
        print("\tExpected False, but got True for word: '"+ word +"' and hand:", hand)

        failure = True    

    # dictionary of words and scores WITH wildcards
    words = {("h*ney", 7):290, ("c*ws", 6):176, ("wa*ls", 7):203}
    for (word, n) in words.keys():
        score = get_word_score(word, n)
        if score != words[(word, n)]:
            print("FAILURE: test_get_word_score() with wildcards")
            print("\tExpected", words[(word, n)], "points but got '" + \
                  str(score) + "' for word '" + word + "', n=" + str(n))
            failure=True      

    if not failure:
        print("SUCCESS: test_wildcard()")

In [ ]:
is_valid_word(word = 'h*ney', hand = {'n': 1, 'h': 1, '*': 1, 'y': 1, 'd': 1, 'w': 1, 'e': 2}, wordlist = wordlist)

True

In [ ]:
test_wildcard(wordlist)

SUCCESS: test_wildcard()


###Problem 5

In [ ]:
def calculate_handlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string-> int)
    returns: integer
    """
    hand_values = list(hand.values())
    return sum(hand_values)

###Problem 6

In [ ]:
def substitute_hand(hand, letter):
    """ 
    Allow the user to replace all copies of one letter in the hand (chosen by user)
    with a new letter chosen from the VOWELS and CONSONANTS at random. The new letter
    should be different from user's choice, and should not be any of the letters
    already in the hand.

    If user provide a letter not in the hand, the hand should be the same.

    Has no side effects: does not mutate hand.

    For example:
        substitute_hand({'h':1, 'e':1, 'l':2, 'o':1}, 'l')
    might return:
        {'h':1, 'e':1, 'o':1, 'x':2} -> if the new letter is 'x'
    The new letter should not be 'h', 'e', 'l', or 'o' since those letters were
    already in the hand.
    
    hand: dictionary (string -> int)
    letter: string
    returns: dictionary (string -> int)
    """
    letter_low = letter.lower()
    sub_hand = hand.copy()
    if letter_low not in hand.keys():
      return sub_hand
    else:
      val = sub_hand[letter_low]
      del sub_hand[letter_low]
      available_letters = (VOWELS + CONSONANTS).replace(letter_low, '')
      for char in ' '.join(list(hand.keys())):
        available_letters = available_letters.replace(char, '')
      random_letter = random.choice(available_letters)
      sub_hand[random_letter] = val
      return sub_hand


In [ ]:
substitute_hand({'h':1, 'e':1, 'l':2, 'o':1}, 'l')

{'e': 1, 'h': 1, 'm': 2, 'o': 1}

#Final game

In [ ]:
def play_hand(hand, word_list):

    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    
    * The user may input a word.

    * When any word is entered (valid or invalid), it uses up letters
      from the hand.

    * An invalid word is rejected, and a message is displayed asking
      the user to choose another word.

    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.

    * The sum of the word scores is displayed when the hand finishes.

    * The hand finishes when there are no more unused letters.
      The user can also finish playing the hand by inputing two 
      exclamation points (the string '!!') instead of a word.

      hand: dictionary (string -> int)
      word_list: list of lowercase strings
      returns: the total score for the hand
      
    """
    
    # BEGIN PSEUDOCODE <-- Remove this comment when you implement this function
    # Keep track of the total score
    hand_total_score = 0 
    print('Current hand is: {}'.format(hand))
    sub_question = str(input('Would you like to substitute a letter? Please, enter yes or no: ')).lower()
    if sub_question == 'yes':
      letter = str(input('Please enter the letter to substitute: ')).lower()
      hand = substitute_hand(hand, letter)
    else:
      pass
    while calculate_handlen(hand) > 0:
      print('Current hand is: {}'.format(hand))
      input_word = str(input('Enter word, or "!!" to indicate that you are finished: ')).lower()
      if input_word == '!!':
        break
      else:
        if (is_valid_word(input_word, hand, wordlist)) == True:
          current_score = get_word_score(input_word, calculate_handlen(hand))
          hand_total_score += current_score
          hand = update_hand(hand, input_word)
          print('{word} earns {current_score}'.format(word = input_word, current_score = current_score))
          print('Total score: {} points'.format(hand_total_score))
        else:
          print('The word is invalid')
          print('Total score: {} points'.format(hand_total_score))
          hand = update_hand(hand, input_word)
    if calculate_handlen(hand) > 0:
      print('Total score for this hand is: {} points'.format(hand_total_score))
      return hand_total_score
    else:
      print('Ran out of letters.')
      print('Total score for this hand is {}'.format(hand_total_score))
      return hand_total_score
    
 
    # As long as there are still letters left in the hand:
    
        # Display the hand
        
        # Ask user for input
         # If the input is two exclamation points:
        
            # End the game (break out of the loop)

            
        # Otherwise (the input is not two exclamation points):

            # If the word is valid:

                # Tell the user how many points the word earned,
                # and the updated total score

            # Otherwise (the word is not valid):
                # Reject invalid word (print a message)
                
            # update the user's hand by removing the letters of their inputted word
            

    # Game is over (user entered '!!' or ran out of letters),
    # so tell user the total score

    # Return the total score as result of function

In [ ]:
def play_game(word_list, hand_size):
    """
    Allow the user to play a series of hands

    * Asks the user to input a total number of hands

    * Accumulates the score for each hand into a total score for the 
      entire series
 
    * For each hand, before playing, ask the user if they want to substitute
      one letter for another. If the user inputs 'yes', prompt them for their
      desired letter. This can only be done once during the game. Once the
      substitue option is used, the user should not be asked if they want to
      substitute letters in the future.

    * For each hand, ask the user if they would like to replay the hand.
      If the user inputs 'yes', they will replay the hand and keep 
      the better of the two scores for that hand.  This can only be done once 
      during the game. Once the replay option is used, the user should not
      be asked if they want to replay future hands. Replaying the hand does
      not count as one of the total number of hands the user initially
      wanted to play.

            * Note: if you replay a hand, you do not get the option to substitute
                    a letter - you must play whatever hand you just had.
      
    * Returns the total score for the series of hands

    word_list: list of lowercase strings
    """
    
    number_of_hands = int(input('Enter total number of hands: '))
    total_score = 0
    for i in range(number_of_hands):
      hand = deal_hand(hand_size)
      hand_score = play_hand(hand, wordlist)
      total_score += hand_score
      replay_question = str(input('Would you like to replay the hand? Please, enter yes or no: ')).lower()
      if replay_question == 'yes':
        total_score -= hand_score 
        hand_score = play_hand(hand, wordlist)
        total_score += hand_score 
      else:
        pass
        
    print('---------')
    print('Total score over all hands: {}'.format(total_score))




###Test

In [ ]:
play_game(wordlist, 6)

Enter total number of hands: 2
Current hand is: {'o': 1, '*': 1, 'z': 1, 'n': 2, 'l': 1}
Would you like to substitute a letter? Please, enter yes or no: no
Current hand is: {'o': 1, '*': 1, 'z': 1, 'n': 2, 'l': 1}
Enter word, or "!!" to indicate that you are finished: zol
The word is invalid
Total score: 0 points
Current hand is: {'o': 0, '*': 1, 'z': 0, 'n': 2, 'l': 0}
Enter word, or "!!" to indicate that you are finished: nn1
The word is invalid
Total score: 0 points
Current hand is: {'o': 0, '*': 1, 'z': 0, 'n': 0, 'l': 0}
Enter word, or "!!" to indicate that you are finished: *
The word is invalid
Total score: 0 points
Ran out of letters.
Total score for this hand is 0
Would you like to replay the hand? Please, enter yes or no: no
Current hand is: {'e': 1, '*': 1, 's': 1, 't': 1, 'd': 1, 'r': 1}
Would you like to substitute a letter? Please, enter yes or no: yes
Please enter the letter to substitute: s
Current hand is: {'e': 1, '*': 1, 't': 1, 'd': 1, 'r': 1, 'l': 1}
Enter word, or